In [9]:
# !uv pip install --system numexpr

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")

In [4]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [5]:
from langchain.tools import DuckDuckGoSearchResults
from langchain.agents import load_tools, Tool

In [10]:
# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool=Tool(
    name='duckduck',
    description="A web search engine. Use this as a search engine for general queires.",
    func=search.run,
    )

# perpare tool
tools=load_tools(["llm-math"], llm=gemini_llm)
tools.append(search_tool)

In [11]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent 
agent = create_react_agent(gemini_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [12]:
agent_executor.invoke({
    "input": "What is the current price of a MacBook Pro in USD? Howmuch would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
})



> Entering new AgentExecutor chain...
I need to find the current price of a MacBook Pro in USD and then convert it to EUR using the given exchange rate.
Action: duckduck
Action Input: "current price of MacBook Pro in USD"snippet: The 14-inch M4 MacBook Pro with 16GB RAM and 1TB comes with $316 off, bringing the price down from $1,799 to $1,483, the best price we've seen to date., title: This Week's Best Apple Deals: M4 MacBook Pro $509 Off, M3 iMac $400 Off ..., link: https://www.macobserver.com/deals/this-weeks-best-apple-deals-m4-macbook-pro-509-off-m3-imac-400-off-and-more/, snippet: Find the cheapest prices and best deals on current MacBook Pros at AppleInsider., title: Best MacBook Pro Deals for June 2025 | Save up to $500 - AppleInsider, link: https://appleinsider.com/deals/best-macbook-pro-deals, snippet: Finally, the MacBook Pro's M4 Pro processor crushed performance tests and scoffed at everything we threw at it. It landed an astonishing score of 22,822 in Geekbench 6.3's ov

{'input': 'What is the current price of a MacBook Pro in USD? Howmuch would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?',
 'output': 'The current starting price of a 16-inch MacBook Pro is $2,499 USD. At an exchange rate of 0.85 EUR per 1 USD, it would cost 2124.15 EUR.'}